In [2]:
import pandas as pd
import prediction_of_H_indicator_with_subCatchmentData as prediction
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [3]:
RESULT_FOLDER = "output"
NB_CLUSTERS=10

In [4]:
input_data_cluster = pd.read_csv(RESULT_FOLDER + "/" + "clusters_categories" + str(NB_CLUSTERS) + "_cut.csv", sep=",")

In [5]:
input_data_cluster

,Site,SubCatch,Category,clusters
0,1,1,D,2
1,1,3,D,9
2,1,4,D,2
3,1,5,P,0
4,1,6,D,0
...,...,...,...,...
219,39,4,G,6
220,39,5,P,8
221,39,7,G,7
222,39,8,P,8


In [12]:
def get_cluster_for_site_subcath(input_data_cluster, site_number, subcatch_number):
    return int(input_data_cluster.loc[(input_data_cluster.Site == site_number) & (input_data_cluster.SubCatch == subcatch_number)]["clusters"])

In [13]:

subcatch_in_clusters = {}
for cluster in range(NB_CLUSTERS):
    subcatch_in_clusters[cluster] = {}
    data_for_cluster = input_data_cluster[input_data_cluster["clusters"]==cluster]
    #print(data_for_cluster)
    for index in range(len(data_for_cluster)):
        site = data_for_cluster.iloc[index, 0]
        sub = data_for_cluster.iloc[index, 1]
        if site not in subcatch_in_clusters[cluster]:
            subcatch_in_clusters[cluster][site] = [sub]
        else:
            subcatch_in_clusters[cluster][site].append(sub)

In [14]:
input_data_cluster

,Site,SubCatch,Category,clusters
0,1,1,D,2
1,1,3,D,9
2,1,4,D,2
3,1,5,P,0
4,1,6,D,0
...,...,...,...,...
219,39,4,G,6
220,39,5,P,8
221,39,7,G,7
222,39,8,P,8


In [15]:
input_data = prediction.import_input_data()
input_data_cleaned = prediction.clean_data(input_data)
input_data_cleaned

,Site,SubCatch,Slope,Elevation,LC,SAR,Area,CV,HV,HError,Rate
0,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,0.000000,1.0
1,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,0.003310,2.0
2,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,0.003310,7.0
3,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,0.003310,15.0
4,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,0.035656,21.0
...,...,...,...,...,...,...,...,...,...,...,...
6784,39,9,5.605823,217.943283,640.819977,1.001974,5394375,0,26,0.003682,2000.0
6785,39,9,5.605823,217.943283,640.819977,1.001974,5394375,0,26,0.003682,2250.0
6786,39,9,5.605823,217.943283,640.819977,1.001974,5394375,0,26,0.003682,3000.0
6787,39,9,5.605823,217.943283,640.819977,1.001974,5394375,0,26,0.003682,3182.0


In [16]:
result = pd.merge(input_data_cleaned, input_data_cluster, how="left",on=["Site", "SubCatch"])
result

,Site,SubCatch,Slope,Elevation,LC,SAR,Area,CV,HV,HError,Rate,Category,clusters
0,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,0.000000,1.0,D,2.0
1,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,0.003310,2.0,D,2.0
2,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,0.003310,7.0,D,2.0
3,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,0.003310,15.0,D,2.0
4,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,0.035656,21.0,D,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6309,39,9,5.605823,217.943283,640.819977,1.001974,5394375,0,26,0.003682,2000.0,P,9.0
6310,39,9,5.605823,217.943283,640.819977,1.001974,5394375,0,26,0.003682,2250.0,P,9.0
6311,39,9,5.605823,217.943283,640.819977,1.001974,5394375,0,26,0.003682,3000.0,P,9.0
6312,39,9,5.605823,217.943283,640.819977,1.001974,5394375,0,26,0.003682,3182.0,P,9.0


In [17]:
site_number = 1
test_subcatch = 3
cluster_number = get_cluster_for_site_subcath(input_data_cluster, site_number, test_subcatch)
data_cluster = result.loc[result["clusters"]==cluster_number]

features, variable_to_predict = prediction.split_dataset_into_features_and_variable_to_predict(data_cluster)
print(features, variable_to_predict)

      Site  SubCatch     Slope   Elevation          LC       SAR     Area  CV  \
30       1         3  3.146334   46.627327  666.589392  1.000757  6120000   0   
31       1         3  3.146334   46.627327  666.589392  1.000757  6120000   0   
32       1         3  3.146334   46.627327  666.589392  1.000757  6120000   0   
33       1         3  3.146334   46.627327  666.589392  1.000757  6120000   0   
34       1         3  3.146334   46.627327  666.589392  1.000757  6120000   0   
...    ...       ...       ...         ...         ...       ...      ...  ..   
6309    39         9  5.605823  217.943283  640.819977  1.001974  5394375   0   
6310    39         9  5.605823  217.943283  640.819977  1.001974  5394375   0   
6311    39         9  5.605823  217.943283  640.819977  1.001974  5394375   0   
6312    39         9  5.605823  217.943283  640.819977  1.001974  5394375   0   
6313    39         9  5.605823  217.943283  640.819977  1.001974  5394375   0   

       HV    Rate Category 

In [18]:
def get_train_and_test_data(X, y, test_site, test_subcatch):
    y_test = y.loc[(y.Site == test_site) & (y.SubCatch == test_subcatch)]
        ## We do not want to take the site number into account for the prediction
    del y_test["Site"]
    del y_test["SubCatch"]

        # Removing the data for the site we want to predict  =>>> /!\ TO DO : remove only the test subcatch of the test site ?
    y_train = y.drop(y[y.Site == test_site].index)
        ## We do not want to take the site number into account for the prediction
    del y_train["Site"]
    del y_train["SubCatch"]

        # Splitting the x (features) into training and testing data
    X_test = X.loc[(X.Site == test_site) & (X.SubCatch == test_subcatch)]
        ## We do not want to take the site number into account for the prediction
    del X_test["Site"]
    del X_test["SubCatch"]
    del X_test["Category"]
    del X_test["clusters"]
        # Removing the data for the site we want to predict
    X_train = X.drop(X[X.Site == test_site].index)
        ## We do not want to take the site number into account for the prediction
    del X_train["Site"]
    del X_train["SubCatch"]
    del X_train["Category"]
    del X_train["clusters"]
    
    return X_train, X_test, y_train, y_test

In [19]:
features_train, features_test, variable_train, variable_test = get_train_and_test_data(features, variable_to_predict, site_number, test_subcatch)

In [20]:
def train_random_forest_model(X_train, y_train):
    forest = RandomForestRegressor(
        n_estimators=1000, criterion="mse", random_state=1, n_jobs=-1, oob_score = True, bootstrap = True
    )
    forest.fit(X_train, y_train.values.ravel())
    return forest

prediction_model = train_random_forest_model(features_train, variable_train)

In [21]:
def predict_with_trained_model(forest, X_train, X_test):
    # Predicting results
    y_train_pred = forest.predict(X_train)
    y_test_pred = forest.predict(X_test)
    return y_train_pred, y_test_pred

variable_train_pred, variable_test_pred = predict_with_trained_model(prediction_model, features_train, features_test)

In [22]:
#subCatchment_numbers = prediction.get_subcatchment_numbers_for_a_site(site_number, variable_to_predict)

#print(subCatchment_numbers)
liste_variable_test_HError = prediction.get_list_variable_test_Hind_Values(variable_test)
subCatchment_numbers = [test_subcatch] * len((variable_test))
#print(variable_test)
#print(liste_variable_test_HError)
liste_variable_test_pred_HError = variable_test_pred
mse_test, r2_test = prediction.get_standard_quality_metrics(subCatchment_numbers, liste_variable_test_HError, liste_variable_test_pred_HError)

In [23]:
rates = prediction.get_rates_for_a_site(site_number, features)
pmax_test, pmax_pred = prediction.get_real_and_pred_pmax(subCatchment_numbers, rates, liste_variable_test_HError, variable_test_pred)

Real value of p:  {3: 100.0}
Predicted value of p:  {3: 125.0}


In [24]:
prediction.save_Hind_results_into_file(site_number, subCatchment_numbers, rates, liste_variable_test_HError, variable_test_pred, approx=0, chronicle=0, permeability=27.32)
prediction.save_Pmax_results_into_file(site_number, pmax_test, pmax_pred, mse_test, r2_test, approx=0, chronicle=0, permeability=27.32)

File created here:  output/Approx0/Chronicle0/SiteTest1/Prediction_HErrorValues_SubCatch_Chronicle0_Approx0_K27.32_Slope_Elevation_LC_SAR_Area_CV_HV.csv
File created here: output/Approx0/Chronicle0/SiteTest1/Prediction_PMax_SubCatch_Chronicle0_Approx0_K27.32_Slope_Elevation_LC_SAR_Area_CV_HV.csv


In [103]:
import pandas as pd
import numpy as np
import csv
import os
OUTPUT_FOLDER = "output"

def save_Hind_results_into_file(site_number, subcatch_number, subCatchment_numbers, rates, liste_y_test_HError, y_test_pred, approx=0, chronicle=0, permeability=27.32):

    MYDIR = (
    OUTPUT_FOLDER
    + "/"
    + "Approx"
    + str(approx)
    + "/Chronicle"
    + str(chronicle)
    + "/SiteTest"
    + str(site_number)
    )

    # Checking if the path and directory where to store the prediction files exists, if not it is created
    CHECK_FOLDER = os.path.isdir(MYDIR)
    if not CHECK_FOLDER:
        os.makedirs(MYDIR)
        print("The directory did not exist. It has been created here: " + MYDIR)

    with open(
        MYDIR
        + "/"
        + "Prediction_HErrorValues_SubCatch"
        + str(subcatch_number)
        + "_by_cluster_Chronicle"
        + str(chronicle)
        + "_Approx"
        + str(approx)
        + "_K"
        + str(permeability)
        + "_Slope_Elevation_LC_SAR_Area_CV_HV.csv",
        "w",
    ) as f:
        writer = csv.writer(f, delimiter=";")
        writer.writerow(
            [
                "Approx",
                "Chronicle",
                "Test Site",
                "SubCatchment",
                "Rate",
                "H Error Real",
                "H Error Predict",
            ]
        )
        for i in range(len(liste_y_test_HError)):
            writer.writerow(
                [
                    approx,
                    chronicle,
                    site_number,
                    subCatchment_numbers[i],
                    rates[i],
                    liste_y_test_HError[i],
                    y_test_pred[i],
                ]
            )
    print("File created here: ", MYDIR
        + "/"
        + "Prediction_HErrorValues_SubCatch"
        + str(subcatch_number)
        + "_Chronicle"
        + str(chronicle)
        + "_Approx"
        + str(approx)
        + "_K"
        + str(permeability)
        + "_Slope_Elevation_LC_SAR_Area_CV_HV.csv")

def save_Pmax_results_into_file(site_number, subcatch_number, p_test, p_pred, mse_test, r2_test, approx=0, chronicle=0, permeability=27.32):
    MYDIR = (
    OUTPUT_FOLDER
    + "/"
    + "Approx"
    + str(approx)
    + "/Chronicle"
    + str(chronicle)
    + "/SiteTest"
    + str(site_number)
    )
    
    # Checking if the path and directory where to store the prediction files exists, if not it is created
    CHECK_FOLDER = os.path.isdir(MYDIR)
    if not CHECK_FOLDER:
        os.makedirs(MYDIR)
        print("The directory did not exist. It has been created here: " + MYDIR)
    
    with open(
        MYDIR
        + "/"
        + "Prediction_PMax_SubCatch"
        + str(subcatch_number)
        + "_by_cluster_Chronicle"
        + str(chronicle)
        + "_Approx"
        + str(approx)
        + "_K"
        + str(permeability)
        + "_Slope_Elevation_LC_SAR_Area_CV_HV.csv",
        "w",
    ) as f:
        writer = csv.writer(f, delimiter=";")
        writer.writerow(
            [
                "Approx",
                "Chronicle",
                "Test Site",
                "SubCatchment",
                "MSE Test",
                "R2 Test",
                "P Real",
                "P pred",
            ]
        )
        for subCatch in p_test:
            writer.writerow(
                [
                    approx,
                    chronicle,
                    site_number,
                    subCatch,
                    mse_test[subCatch],
                    r2_test[subCatch],
                    p_test[subCatch],
                    p_pred[subCatch],
                ]
            )
    print("File created here: " + MYDIR+ "/"+ "Prediction_PMax_SubCatch" + str(subcatch_number) + "_by_cluster_Chronicle"+ str(chronicle)+ "_Approx" + str(approx)+ "_K"+ str(permeability)+ "_Slope_Elevation_LC_SAR_Area_CV_HV.csv")

In [104]:
site_number = 1
test_subcatch = 1


    cluster_number = get_cluster_for_site_subcath(input_data_cluster, site_number, test_subcatch)
    data_cluster = result.loc[result["clusters"]==cluster_number]

    features, variable_to_predict = prediction.split_dataset_into_features_and_variable_to_predict(data_cluster)
    features_train, features_test, variable_train, variable_test = get_train_and_test_data(features, variable_to_predict, site_number, test_subcatch)
    prediction_model = train_random_forest_model(features_train, variable_train)
    variable_train_pred, variable_test_pred = predict_with_trained_model(prediction_model, features_train, features_test)
    liste_variable_test_HError = prediction.get_list_variable_test_Hind_Values(variable_test)
    subCatchment_numbers = [test_subcatch] * len((variable_test))
    liste_variable_test_pred_HError = variable_test_pred
    mse_test, r2_test = prediction.get_standard_quality_metrics(subCatchment_numbers, liste_variable_test_HError, liste_variable_test_pred_HError)
    rates = prediction.get_rates_for_a_site(site_number, features)
    pmax_test, pmax_pred = prediction.get_real_and_pred_pmax(subCatchment_numbers, rates, liste_variable_test_HError, variable_test_pred)
    save_Hind_results_into_file(site_number, test_subcatch, subCatchment_numbers, rates, liste_variable_test_HError, variable_test_pred, approx=0, chronicle=0, permeability=27.32)
    save_Pmax_results_into_file(site_number, test_subcatch, pmax_test, pmax_pred, mse_test, r2_test, approx=0, chronicle=0, permeability=27.32)



Real value of p:  {1: 90.0}
Predicted value of p:  {1: 550.0}
File created here:  output/Approx0/Chronicle0/SiteTest1/Prediction_HErrorValues_SubCatch1_Chronicle0_Approx0_K27.32_Slope_Elevation_LC_SAR_Area_CV_HV.csv
File created here: output/Approx0/Chronicle0/SiteTest1/Prediction_PMax_SubCatch1_by_cluster_Chronicle0_Approx0_K27.32_Slope_Elevation_LC_SAR_Area_CV_HV.csv
